synthetic data generation
Since the exam allows synthetic data and specifies a structure similar to the UCI Online Retail dataset, I used the `Faker` library to generate a realistic retail dataset with 1000 rows. Each row simulates a transaction and includes the following fields:

InvoiceNo: Random invoice IDs
StockCode: Product codes
Description: Product names
Quantity: 1–50
InvoiceDate: Random dates over 2 years
UnitPrice: 1–100
CustomerID: 100 unique customers
Country: 5–10 countries

The data is stored in a pandas DataFrame and exported to `synthetic_data.csv`, which is used as input for the ETL pipeline.

In [18]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker()
Faker.seed(42)
random.seed(42)

# Parameters
num_rows = 1000
num_customers = 100
countries = ['UK', 'Germany', 'France', 'Spain', 'Italy', 'Netherlands', 'Belgium', 'Sweden', 'Norway', 'Denmark']

# Generate data
data = []
for _ in range(num_rows):
    invoice_no = f"{random.randint(100000, 999999)}"
    stock_code = f"P{random.randint(1000, 9999)}"
    description = fake.word().capitalize() + " " + fake.word().capitalize()
    quantity = random.randint(1, 50)
    unit_price = round(random.uniform(1.0, 100.0), 2)
    invoice_date = fake.date_between(start_date='-2y', end_date='today')
    customer_id = random.randint(10000, 10000 + num_customers)
    country = random.choice(countries)

    data.append([
        invoice_no, stock_code, description, quantity,
        invoice_date, unit_price, customer_id, country
    ])

# Create DataFrame
df = pd.DataFrame(data, columns=[
    "InvoiceNo", "StockCode", "Description", "Quantity",
    "InvoiceDate", "UnitPrice", "CustomerID", "Country"
])

# Save to CSV
df.to_csv("synthetic_data.csv", index=False)
print("Synthetic dataset saved as synthetic_data.csv")


Synthetic dataset saved as synthetic_data.csv


1. Extract
Load your synthetic_data.csv into a pandas DataFrame
Convert InvoiceDate to datetime
Handle missing values (e.g., drop or fill)

In [19]:
df = pd.read_csv("synthetic_data.csv")
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df.dropna(inplace=True)


2. Transform
Add TotalSales = Quantity * UnitPrice
Filter for sales in the last year (from August 12, 2024 to August 12, 2025)
Remove outliers: Quantity < 0 or UnitPrice <= 0
Create dimension-like summaries (e.g., group by CustomerID)

In [20]:
df["TotalSales"] = df["Quantity"] * df["UnitPrice"]
df = df[df["InvoiceDate"] >= "2024-08-12"]
df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]

customer_summary = df.groupby("CustomerID").agg({
    "TotalSales": "sum",
    "Country": "first"
}).reset_index()


3. Load
Use sqlite3 to create retail_dw.db
Create tables: SalesFact, CustomerDim, TimeDim
Insert transformed data

In [21]:
import sqlite3

conn = sqlite3.connect("retail_dw.db")
df.to_sql("SalesFact", conn, if_exists="replace", index=False)
customer_summary.to_sql("CustomerDim", conn, if_exists="replace", index=False)
# You can also extract year/month from InvoiceDate for TimeDim


99

4. Wrap in a Function with Logging
Log number of rows at each stage
Handle errors gracefully

In [22]:
def run_etl():
    try:
        print("Extracting data...")
        df = pd.read_csv("etl/synthetic_data.csv")
        print(f"Rows extracted: {len(df)}")

        # Transform
        df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
        df["TotalSales"] = df["Quantity"] * df["UnitPrice"]
        df = df[df["InvoiceDate"] >= "2024-08-12"]
        df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]
        print(f"Rows after cleaning: {len(df)}")

        # Load
        conn = sqlite3.connect("retail_dw.db")
        df.to_sql("SalesFact", conn, if_exists="replace", index=False)
        print("Data loaded into SalesFact table.")

    except Exception as e:
        print(f"ETL failed: {e}")
